In [1]:
import os
import time
import numpy as np

from model import model_train
from model import model_load
from model import model_predict

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from cslib import fetch_ts, engineer_features

In [2]:
base_dir = os.getcwd()
DATA_DIR = os.path.join(base_dir , "data" , "cs-train")

ts_all = fetch_ts(DATA_DIR,clean=False)

... loading ts data from files


In [3]:
X,y,dates = engineer_features(ts_all['all'])
        
## Perform a train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)

In [6]:
# Evaluate RandomForestRegressor

eval_model = 'RandomForestRegressor'

param_grid_rf = {
    'rf__criterion': ['mse','mae'],
    'rf__n_estimators': [10,15,20,25,50,100]
    }

time_start = time.time()
pipe_rf = Pipeline(steps=[('scaler', StandardScaler()), ('rf', RandomForestRegressor())])

grid = GridSearchCV(pipe_rf, param_grid=param_grid_rf, cv=5, iid=False, n_jobs=-1)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

rf_mae =  mean_absolute_error(y_test, y_pred)
rf_mse =  mean_squared_error(y_test, y_pred)
rf_r2_score = r2_score(y_test, y_pred)

print("train time = ", time.strftime('%H:%M:%S', time.gmtime(time.time()-time_start)))
print("mae for {} = {:.0f}".format(eval_model, rf_mae))
print("mse for {} = {:.0f}".format(eval_model, rf_mse))
print("r2_score for {} = {:.3f}".format(eval_model, rf_r2_score))
print("best params for {} = {}".format(eval_model, grid.best_params_))

train time =  00:00:06
mae for RandomForestRegressor = 11335
mse for RandomForestRegressor = 257143480
r2_score for RandomForestRegressor = 0.961
best params for RandomForestRegressor = {'rf__criterion': 'mse', 'rf__n_estimators': 50}


In [7]:
# Evaluate GradientBoostingRegressor

eval_model = 'GradientBoostingRegressor'

param_grid_gb = {
    'gb__criterion': ['mse','mae'],
    'gb__n_estimators': [10,15,20,25,50,100]
    }

time_start = time.time()
pipe_gb = Pipeline(steps=[('scaler', StandardScaler()), ('gb', GradientBoostingRegressor())])

grid = GridSearchCV(pipe_gb, param_grid=param_grid_gb, cv=5, iid=False, n_jobs=-1)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

gb_mae =  mean_absolute_error(y_test, y_pred)
gb_mse =  mean_squared_error(y_test, y_pred)
gb_r2_score = r2_score(y_test, y_pred)

print("train time = ", time.strftime('%H:%M:%S', time.gmtime(time.time()-time_start)))
print("mae for {} = {:.0f}".format(eval_model, gb_mae))
print("mse for {} = {:.0f}".format(eval_model, gb_mse))
print("r2_score for {} = {:.3f}".format(eval_model, gb_r2_score))
print("best params for {} = {}".format(eval_model, grid.best_params_))

train time =  00:00:05
mae for GradientBoostingRegressor = 16151
mse for GradientBoostingRegressor = 450761320
r2_score for GradientBoostingRegressor = 0.931
best params for GradientBoostingRegressor = {'gb__criterion': 'mse', 'gb__n_estimators': 100}


In [8]:
# Evaluate DecisionTreeRegressor

eval_model = 'DecisionTreeRegressor'

param_grid_dt = {
    'dt__criterion': ['mse','mae'],
    'dt__max_depth': [5,10,20,50],
    'dt__min_samples_leaf': [1,2,3,4,5]
    }

time_start = time.time()
pipe_ts = Pipeline(steps=[('scaler', StandardScaler()), ('dt', DecisionTreeRegressor())])

grid = GridSearchCV(pipe_ts, param_grid=param_grid_dt, cv=5, iid=False, n_jobs=-1)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

dt_mae =  mean_absolute_error(y_test, y_pred)
dt_mse =  mean_squared_error(y_test, y_pred)
dt_r2_score = r2_score(y_test, y_pred)

print("train time = ", time.strftime('%H:%M:%S', time.gmtime(time.time()-time_start)))
print("mae for {} = {:.0f}".format(eval_model, dt_mae))
print("mse for {} = {:.0f}".format(eval_model, dt_mse))
print("r2_score for {} = {:.3f}".format(eval_model, dt_r2_score))
print("best params for {} = {}".format(eval_model, grid.best_params_))

train time =  00:00:01
mae for DecisionTreeRegressor = 12375
mse for DecisionTreeRegressor = 439838827
r2_score for DecisionTreeRegressor = 0.933
best params for DecisionTreeRegressor = {'dt__criterion': 'mse', 'dt__max_depth': 10, 'dt__min_samples_leaf': 2}
